<a href="https://colab.research.google.com/github/Battula888/FMML-LAB/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging the validation accuracy across multiple splits generally gives more consistent and reliable results. This approach, often referred to as cross-validation, helps to mitigate the variability that can arise from the specific choice of a single validation set. Here's why:

Reduction in Variance: A single split of the data may not be representative of the overall distribution, and it could result in higher variance in the performance metrics. Averaging across multiple splits smooths out any idiosyncratic trends or biases present in a single validation set.

Better Generalization: By using multiple different subsets of the data for validation, you ensure that your model's performance is evaluated on various portions of the data, improving the estimate of how well the model generalizes to unseen data.

Minimizing Overfitting to the Validation Set: In a single split, there’s a risk of the model overfitting to the validation set, particularly if the model is complex. By averaging over multiple splits, you reduce the likelihood that your model is overly tailored to a specific subset of the data.

How it works:
In k-fold cross-validation, for instance, the data is divided into k parts (or folds). The model is trained on k-1 folds and tested on the remaining fold. This process is repeated k times, with each fold serving as the validation set once. The final result is the average performance metric (such as accuracy) across all the splits.

Example:
Without averaging: If you evaluate on a single validation set, the accuracy might be 85%, but this could be a result of lucky or unlucky splits.
With averaging: By running k-fold cross-validation (say 5-fold), you might get validation accuracies of 80%, 83%, 87%, 85%, and 90%. Averaging these gives a more reliable and consistent estimate of model performance (85% in this case), reducing the influence of a single outlier.
Conclusion:
Averaging validation accuracy across multiple splits helps provide a more stable and consistent evaluation of your model's performance,

2.DOES it give more accurate estimate of test accuracy?
Averaging the validation accuracy across multiple splits does not directly give a **more accurate estimate of test accuracy**, but it does provide a **better estimate of how well the model is likely to perform on unseen data**. Here's a breakdown:

### Key Differences Between Validation and Test Accuracy

- **Validation Accuracy**: This is the accuracy of your model on a separate validation set (a subset of the training data) during training. Cross-validation helps improve the reliability of this estimate by averaging across multiple splits of the data.
- **Test Accuracy**: This is the performance of your model on a completely separate test set that the model has never seen during training or validation. The test set is meant to simulate the performance of your model on truly unseen data.

### Does Averaging Validation Accuracy Improve the Estimate of Test Accuracy?

- **Indirect Improvement**: By using multiple validation splits (such as in cross-validation), you get a better estimate of your model's **generalization ability** because it reduces the risk of overfitting to a specific validation set. A more reliable validation accuracy can help you choose a model or hyperparameters that are more likely to perform well on the test set.
  
  However, this does **not** directly provide a more accurate estimate of the test set performance itself. The test set is a completely separate entity that ideally should not be used during training or validation.

- **Overfitting Mitigation**: Cross-validation helps reduce the risk of overfitting to the validation set, which in turn may make the model more likely to generalize well to the test set. This makes the **validation accuracy** a more reliable indicator of how the model might perform on the test set. However, even with cross-validation, the true test performance can still differ slightly, especially if the test set is small or not perfectly representative.

### When You Get More Accurate Test Accuracy Estimates:
- **External Testing**: The most accurate estimate of test accuracy comes when you have a separate, untouched test set. If you're using cross-validation to tune models or select features, this can give you confidence that your model is likely to perform well on the test set, but you won't know the true performance on the test set until you evaluate it.

- **Large Datasets**: In scenarios with large amounts of data, cross-validation can give a better estimate of the test accuracy because each fold in the cross-validation uses a different subset of data. In these cases, the validation accuracy can serve as a reasonable proxy for test accuracy.

### Summary:

- **Cross-validation improves the consistency and reliability of your validation accuracy**, which indirectly helps in making better decisions regarding model selection and tuning.
- **Averaging validation accuracy does not directly give a more accurate estimate of test accuracy**, but it does provide a more stable estimate of model performance across different subsets of data. The actual test accuracy will only be known after evaluating the model on a completely separate test set.


3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

When working with datasets that have four or more features, testing or visualizing them can become more complex. Below are several techniques you can use to understand relationships and distributions in such high-dimensional data:

### 1. **Pairwise Scatter Plots (Pair Plot)**
   - **Description**: A pair plot (also known as a scatterplot matrix) plots each pair of features against one another. It helps visualize pairwise relationships between features in a dataset.
   - **Use Case**: Works well when you have multiple features and want to look for correlations or patterns between each pair of features.
   - **Visualization Tools**:
     - **Python libraries**: `seaborn.pairplot()`, `matplotlib` with `scatter()`

   ```python
   import seaborn as sns
   sns.pairplot(data)  # 'data' is your DataFrame
   ```

   - **Pros**: Helps in detecting correlations between pairs of features.
   - **Cons**: Becomes crowded with too many features (e.g., 4 features results in a 4x4 grid).

### 2. **Principal Component Analysis (PCA)**
   - **Description**: PCA is a dimensionality reduction technique that projects the data into fewer dimensions (typically 2 or 3) while retaining most of the variance in the data. PCA can be useful for visualizing high-dimensional data.
   - **Use Case**: When you want to reduce the dimensionality of a dataset (e.g., from 4+ features to 2 or 3) for easy visualization.
   - **Visualization Tools**:
     - **Python libraries**: `sklearn.decomposition.PCA`, `matplotlib`, `seaborn`

   ```python
   from sklearn.decomposition import PCA
   pca = PCA(n_components=2)
   pca_result = pca.fit_transform(data)  # 'data' is your DataFrame
   sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1])
   ```

   - **Pros**: Reduces dimensionality, making it easier to visualize higher-dimensional data.
   - **Cons**: May lose information, as it compresses features into principal components.

### 3. **t-Distributed Stochastic Neighbor Embedding (t-SNE)**
   - **Description**: t-SNE is another dimensionality reduction method that is particularly good at preserving local structure and creating well-separated clusters in 2D or 3D visualizations. It's often used when dealing with complex datasets that have many features.
   - **Use Case**: When you want to visualize complex, high-dimensional data in 2D or 3D and preserve local relationships.
   - **Visualization Tools**:
     - **Python libraries**: `sklearn.manifold.TSNE`, `matplotlib`, `seaborn`

   ```python
   from sklearn.manifold import TSNE
   tsne = TSNE(n_components=2)
   tsne_result = tsne.fit_transform(data)
   sns.scatterplot(x=tsne_result[:, 0], y=tsne_result[:, 1])
   ```

   - **Pros**: Effective for high-dimensional data visualization and cluster identification.
   - **Cons**: Computationally expensive and might be sensitive to hyperparameters.

### 4. **Heatmap (Correlation Matrix)**
   - **Description**: A heatmap of the correlation matrix can be used to visualize the pairwise correlation between features. It can help identify strongly correlated features, which may indicate redundant information or relationships.
   - **Use Case**: To check for correlations or redundancy between features.
   - **Visualization Tools**:
     - **Python libraries**: `seaborn.heatmap()`

   ```python
   import seaborn as sns
   import matplotlib.pyplot as plt

   correlation_matrix = data.corr()
   sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
   plt.show()
   ```

   - **Pros**: Identifies correlations, helpful for feature selection or engineering.
   - **Cons**: Not a direct visualization of feature distribution; only shows relationships.

### 5. **Parallel Coordinates Plot**
   - **Description**: A parallel coordinates plot shows the relationships between multiple features simultaneously. Each feature gets its own vertical axis, and each data point is represented as a line across the axes.
   - **Use Case**: Suitable for visualizing multivariate data, especially when comparing multiple features across different categories or classes.
   - **Visualization Tools**:
     - **Python libraries**: `pandas.plotting.parallel_coordinates`, `matplotlib`

   ```python
   from pandas.plotting import parallel_coordinates
   parallel_coordinates(data, class_column='class')  # 'class' is the column for classification
   ```

   - **Pros**: Shows interactions between many features, good for exploring multivariate data.
   - **Cons**: Can become cluttered with many categories or features.

### 6. **Radar (Spider) Charts**
   - **Description**: Radar charts display multivariate data in a two-dimensional space, with one axis for each feature. This is useful when you want to compare data points in terms of multiple features simultaneously.
   - **Use Case**: Best for comparing a few observations across many features.
   - **Visualization Tools**:
     - **Python libraries**: `matplotlib`

   ```python
   import numpy as np
   import matplotlib.pyplot as plt

   labels = data.columns
   values = data.iloc[0].values  # First row for example

   angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
   values = np.concatenate((values, [values[0]]))
   angles += angles[:1]

   fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
   ax.fill(angles, values, color='blue', alpha=0.25)
   ax.plot(angles, values, color='blue', linewidth=2)
   ax.set_yticklabels([])  # Hide radial labels
   ax.set_xticks(angles[:-1])
   ax.set_xticklabels(labels)
   plt.show()
   ```

   - **Pros**: Good for comparing individual data points or classes across multiple features.
   - **Cons**: Difficult to interpret with too many features.

### 7. **3D Scatter Plot**
   - **Description**: A 3D scatter plot can help visualize relationships among three features at once. While not as intuitive as lower-dimensional plots, it can be useful when exploring interactions between a small set of features.
   - **Use Case**: Works well when you want to see the relationship between exactly three features.
   - **Visualization Tools**:
     - **Python libraries**: `matplotlib`'s `Axes3D`

   ```python
   from mpl_toolkits.mplot3d import Axes3D
   import matplotlib.pyplot as plt

   fig = plt.figure()
   ax = fig.add_subplot(111, projection='3d')
   ax.scatter(data['Feature1'], data['Feature2'], data['Feature3'])
   ax.set_xlabel('Feature1')
   ax.set_ylabel('Feature2')
   ax.set_zlabel('Feature3')
   plt.show()
   ```

   - **Pros**: Direct visualization of three features.
   - **Cons**: Can be challenging to interpret, especially when dealing with more than three features.

### Conclusion:
To test and visualize four or more features effectively, you should consider dimensionality reduction techniques (like PCA, t-SNE), pairwise plots, and correlation matrices to explore relationships. Each visualization technique has strengths depending on your dataset and goals. For high-dimensional datasets, reducing the dimensions while retaining meaningful information is often the best way to make the data comprehensible.


4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Increasing the number of iterations (or epochs) during training is one possible approach to improving model performance when dealing with a small training or validation dataset, but **it is not a guaranteed solution** and should be used with caution. The effectiveness of increasing iterations depends on several factors. Let's explore this concept in detail:

### 1. **Overfitting with Small Datasets**
   - **Problem**: A small training dataset can lead to **overfitting**, where the model becomes overly tailored to the limited data and may not generalize well to unseen data. If the dataset is small, running the model for many iterations may just make it memorize the data, rather than learn the underlying patterns.
   - **Increasing iterations** without any form of regularization or data augmentation can **exacerbate overfitting** because the model will simply over-learn the small dataset, leading to poor generalization.

### 2. **Effect of Increasing Iterations**
   - **Training Longer**: Increasing the number of iterations can help the model to converge to a better local minimum or improve performance by making small adjustments over time. However, this is only effective if the model is not overfitting and the learning process is still meaningful.
   - **Diminishing Returns**: After a certain point, increasing iterations on a small dataset might lead to diminishing returns or even negative effects, such as:
     - The model starts memorizing specific examples rather than learning generalizable patterns.
     - Validation performance may degrade as overfitting increases.
   
### 3. **Better Approaches to Handle Small Datasets**
   Instead of just increasing iterations, here are several strategies that can better help with small datasets:

   - **Regularization**:
     - **L2 Regularization (Ridge)** or **L1 Regularization (Lasso)** can prevent the model from fitting the noise in the dataset by adding a penalty for large weights. This helps control overfitting.
     - **Dropout**: In neural networks, dropout randomly sets a portion of the neurons' outputs to zero, preventing overfitting by making the network less reliant on specific neurons.
   
   - **Data Augmentation**:
     - If you are working with image or text data, applying **data augmentation** (e.g., rotating images, flipping, adding noise) can artificially increase the size of your dataset.
     - For text, augmentation techniques like **synonym replacement** or **back translation** (translating to another language and back) can help create more variations of the same data.
   
   - **Cross-validation**:
     - Use **k-fold cross-validation** to ensure that your model is validated on different subsets of the small dataset, which can help mitigate the issue of overfitting to a single training set.
   
   - **Transfer Learning**:
     - If you have access to a pre-trained model (e.g., for image classification or NLP tasks), you can fine-tune it on your small dataset. This approach leverages knowledge from a large, pre-trained model, reducing the need for a large training dataset.
   
   - **Early Stopping**:
     - Instead of just increasing iterations, monitor the validation performance during training and stop the training when the validation error starts to increase, which is an indication that overfitting is happening.
   
   - **Synthetic Data Generation**:
     - **Generative models**, such as GANs (Generative Adversarial Networks), can be used to generate synthetic examples that mimic the underlying data distribution, thus increasing the size of the dataset.

### 4. **Iterative Learning with Small Datasets**
   - **Fine-Tuning**: For small datasets, you might want to perform more fine-tuned adjustments, such as **learning rate scheduling** (reducing the learning rate over time) to ensure that the model converges in a controlled manner, rather than just increasing iterations.
   - **Batch Size**: Smaller batch sizes might be helpful in training on small datasets as they allow the model to update weights more frequently with each training step, improving convergence.

### 5. **Validation Performance**
   - Increasing iterations without improving generalization might not improve the **validation performance**. Instead, it could result in a growing gap between the training and validation loss, indicating overfitting. It's important to monitor the validation loss and ensure it doesn't diverge as training progresses.

### Conclusion
While increasing the number of iterations can sometimes help a model learn better from the data, **simply increasing iterations on a small dataset is not a comprehensive solution**. Without techniques to combat overfitting, increasing iterations will likely lead to overfitting rather than improved performance. The best approach involves a combination of increasing data through augmentation, using regularization techniques, and leveraging transfer learning or other strategies suited for small datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.